In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, tzinfo
import pytz
from io import StringIO
import os
from IPython.display import Markdown
import boto3

In [2]:
# Wrapper function for convenience
def dm(text):
    return display(Markdown(text))

# Introduction
https://en.youbike.com.tw/region/main/stations/

In [3]:
class ConnectionToS3:
    """Factory method object to create an active boto3 S3 resource. Ensure the correct env variables are set before calling this object.

    Available class method:
        from_env(): create connection from environment variables

    """

    def __init__(
        self,
        bucket_name: str,
        aws_access_key_id: str,
        aws_secret_access_key: str,
        endpoint_url: str = None,
        region_name: str = "ap-northeast-1",
    ):
        self._resource = boto3.resource(
            "s3",
            endpoint_url=endpoint_url,
            region_name=region_name,
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        )
        self._bucket_name = bucket_name

    @classmethod
    def from_env(cls):
        app_env = os.getenv("APP_ENV", "local")
        print("Loading from env: ", app_env)
        if app_env == "local":
            return cls(
                "local-youbike",
                os.environ["MINIO_ACCESS_KEY_ID"],
                os.environ["MINIO_SECRET_ACCESS_KEY"],
                f'http://{os.environ["MINIO_HOST"]}:9000'
            )
        elif app_env == "stage":
            return cls(
                "stage-youbike",
                os.environ["AWS_ACCESS_KEY_ID"],
                os.environ["AWS_SECRET_ACCESS_KEY"],
            )
        else:
            raise Exception(f"The argument env={app_env} is not valid.")

    @property
    def resource(self):
        return self._resource

    @property
    def bucket_name(self):
        return self._bucket_name



In [4]:
os.environ['APP_ENV'] = 'stage'

In [5]:
connection = ConnectionToS3.from_env()
connection.bucket_name

Loading from env:  stage


'stage-youbike'

In [6]:
def download_from_bucket(bucket_name: str, remote_path: str, filter: str, dest_dir: str):
    bucket = connection.resource.Bucket(bucket_name)

    for obj in bucket.objects.all():
        if f'{remote_path}{filter}' in obj.key:
            local_file_path = os.path.join(dest_dir, obj.key)
            if '/' in obj.key:
                os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
                connection.resource.meta.client.download_file(connection.bucket_name, obj.key, local_file_path)
            print(f"Downloaded {obj.key} at {dest_dir}")

In [7]:
def create_or_append_df(parquet_path: str, df: pd.DataFrame) -> pd.DataFrame:
    if df is None:
        concat_df = pd.read_parquet(parquet_path).loc[[]]
    else:
        df_to_append = pd.read_parquet(parquet_path)
        concat_df = pd.concat([df, df_to_append], join='outer')
    return concat_df

In [10]:
download_from_bucket(connection.bucket_name, 'raw_data/', 'youbike_dock_info_2024-03-17', '../tmp_data/parquet_raw_data')

Downloaded raw_data/youbike_dock_info_2024-03-17_00:01:01_raw.parquet at ../tmp_data/parquet_raw_data
Downloaded raw_data/youbike_dock_info_2024-03-17_00:11:00_raw.parquet at ../tmp_data/parquet_raw_data
Downloaded raw_data/youbike_dock_info_2024-03-17_00:20:59_raw.parquet at ../tmp_data/parquet_raw_data
Downloaded raw_data/youbike_dock_info_2024-03-17_00:31:00_raw.parquet at ../tmp_data/parquet_raw_data
Downloaded raw_data/youbike_dock_info_2024-03-17_00:40:53_raw.parquet at ../tmp_data/parquet_raw_data
Downloaded raw_data/youbike_dock_info_2024-03-17_00:50:55_raw.parquet at ../tmp_data/parquet_raw_data
Downloaded raw_data/youbike_dock_info_2024-03-17_01:00:59_raw.parquet at ../tmp_data/parquet_raw_data
Downloaded raw_data/youbike_dock_info_2024-03-17_01:10:53_raw.parquet at ../tmp_data/parquet_raw_data
Downloaded raw_data/youbike_dock_info_2024-03-17_01:20:54_raw.parquet at ../tmp_data/parquet_raw_data
Downloaded raw_data/youbike_dock_info_2024-03-17_01:30:54_raw.parquet at ../tmp_da

In [ ]:
# hist_df.drop(index=hist_df[hist_df['extraction_ts'] > '2024-03-13'].index, inplace=True)

In [11]:
#Create historical data df
hist_df = None
cnt = 0
for i in os.listdir('../tmp_data/parquet_raw_data/raw_data/'):
    if "youbike_dock_info_" in i:
        print(i)
        try: 
            hist_df = create_or_append_df(f"../tmp_data/parquet_raw_data/raw_data/{i}", hist_df)
        except:
            print("failed with ", i)
        cnt += 1
        print(f"Added: {cnt} / {len(os.listdir('../tmp_data/parquet_raw_data/raw_data/'))}")

youbike_dock_info_2024-03-16_11:40:52_raw.parquet
Added: 1 / 433
youbike_dock_info_2024-03-15_12:10:58_raw.parquet
Added: 2 / 433
youbike_dock_info_2024-03-17_18:50:50_raw.parquet
Added: 3 / 433
youbike_dock_info_2024-03-16_16:50:57_raw.parquet
Added: 4 / 433
youbike_dock_info_2024-03-16_20:50:51_raw.parquet
Added: 5 / 433
youbike_dock_info_2024-03-15_20:20:53_raw.parquet
Added: 6 / 433
youbike_dock_info_2024-03-17_07:30:55_raw.parquet
Added: 7 / 433
youbike_dock_info_2024-03-16_13:30:51_raw.parquet
Added: 8 / 433
youbike_dock_info_2024-03-16_16:30:58_raw.parquet
Added: 9 / 433
youbike_dock_info_2024-03-16_08:50:51_raw.parquet
Added: 10 / 433
youbike_dock_info_2024-03-16_02:40:56_raw.parquet
Added: 11 / 433
youbike_dock_info_2024-03-16_18:40:55_raw.parquet
Added: 12 / 433
youbike_dock_info_2024-03-16_05:50:53_raw.parquet
Added: 13 / 433
youbike_dock_info_2024-03-17_11:50:57_raw.parquet
Added: 14 / 433
youbike_dock_info_2024-03-15_09:10:57_raw.parquet
Added: 15 / 433
youbike_dock_info_2

In [12]:
tmp_checkpoint = hist_df.copy(deep=True)

In [96]:
hist_df = tmp_checkpoint.copy(deep=True)

In [13]:
#Enforce Schema for historical df
hist_df = hist_df[['id', 'name', 'type', 'space', 'full', 'empty', 'bike_yb2', 'bike_eyb', 'city', 'area',
         'lat', 'lng', 'address', 'is_open', 'place_id', 'last_update_ts', 'extraction_ts']].reset_index(drop=True)
display(hist_df.head(5), hist_df.shape)

,id,name,type,space,full,empty,bike_yb2,bike_eyb,city,area,lat,lng,address,is_open,place_id,last_update_ts,extraction_ts
0,500108177,潭美公園(南京東路六段482號),2,29,0,29,0,0,台北市,內湖區,25.06092,121.58898,南京東路六段482號,0,NaN,2024-03-15 11:32:19+08:00,2024-03-15 12:10:58+08:00
1,500101233,捷運忠孝復興站(3號出口),2,37,0,37,0,0,台北市,大安區,25.07965,121.54148,忠孝東路四段48號前,0,NaN,2024-03-15 11:28:18+08:00,2024-03-15 12:10:58+08:00
2,500501093,新竹生物醫學園區(南門),2,0,0,0,0,0,新竹縣,竹北市,24.80452,121.04378,文興路二段/高鐵九路口(西北側),0,NaN,2024-03-14 15:50:23+08:00,2024-03-15 12:10:58+08:00
3,500501094,高鐵六路隘口八街口,2,0,0,0,0,0,新竹縣,竹北市,24.80020,121.03900,高鐵六路/隘口八街口(西側),0,NaN,2024-03-14 15:51:15+08:00,2024-03-15 12:10:58+08:00
4,500501090,嘉豐數學公園,2,0,0,0,0,0,新竹縣,竹北市,24.81342,121.02711,嘉豐二街一段/嘉豐一街口(西側),0,NaN,2024-03-14 15:47:14+08:00,2024-03-15 12:10:58+08:00


(3425022, 17)

In [18]:
hist_df_prev = pd.read_parquet('/Users/justinwarambourg/Documents/programming/youbike/eda/youbike_dock_info_history_2024-03-01_2024-03-14_raw.parquet')

In [24]:
concat_df = pd.concat([hist_df_prev, hist_df], join='outer')

In [27]:
concat_df.describe(include='all')

,id,name,type,space,full,empty,bike_yb2,bike_eyb,city,area,lat,lng,address,is_open,place_id,last_update_ts,extraction_ts
count,1.150972e+07,11509718,1.150972e+07,1.150972e+07,1.150972e+07,1.150972e+07,1.150972e+07,1.150972e+07,11329546,11509718,1.150972e+07,1.150972e+07,11509718,1.150972e+07,0.0,11509718,11509718
unique,NaN,6924,NaN,NaN,NaN,NaN,NaN,NaN,12,174,NaN,NaN,7376,NaN,NaN,NaN,NaN
top,NaN,和平公園,NaN,NaN,NaN,NaN,NaN,NaN,新北市,板橋區,NaN,NaN,1,NaN,NaN,NaN,NaN
freq,NaN,5812,NaN,NaN,NaN,NaN,NaN,NaN,2618731,394193,NaN,NaN,20342,NaN,NaN,NaN,NaN
mean,4.360474e+08,NaN,1.870876e+00,2.155736e+01,6.911727e+00,1.349432e+01,5.975820e+00,2.894378e-01,NaN,NaN,2.421688e+01,1.206599e+02,NaN,1.097491e+00,NaN,2024-03-01 08:27:06.986493952+08:00,2024-03-11 18:24:44.865810688+08:00
min,1.002000e+03,NaN,1.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+01,0.000000e+00,0.000000e+00,NaN,NaN,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,2021-06-21 23:47:16+08:00,2024-03-01 04:18:31+08:00
25%,5.001091e+08,NaN,2.000000e+00,1.500000e+01,2.000000e+00,7.000000e+00,1.000000e+00,0.000000e+00,NaN,NaN,2.347192e+01,1.204825e+02,NaN,1.000000e+00,NaN,2024-03-08 00:53:18+08:00,2024-03-08 13:25:29+08:00
50%,5.003060e+08,NaN,2.000000e+00,1.800000e+01,5.000000e+00,1.100000e+01,4.000000e+00,0.000000e+00,NaN,NaN,2.492763e+01,1.212086e+02,NaN,1.000000e+00,NaN,2024-03-12 15:08:20+08:00,2024-03-12 22:00:59+08:00
75%,5.012010e+08,NaN,2.000000e+00,2.600000e+01,9.000000e+00,1.700000e+01,9.000000e+00,0.000000e+00,NaN,NaN,2.502942e+01,1.214829e+02,NaN,1.000000e+00,NaN,2024-03-15 07:12:15+08:00,2024-03-15 11:20:55+08:00
max,5.082010e+08,NaN,2.000000e+00,1.180000e+02,9.900000e+01,9.900000e+01,9.900000e+01,6.200000e+01,NaN,NaN,2.529301e+01,1.220000e+02,NaN,6.000000e+00,NaN,2024-03-17 23:49:18+08:00,2024-03-17 23:50:55+08:00


In [111]:

hist_df['up_tz'] = hist_df['last_update_ts'].transform(lambda x: str(x).split('+')[1])
hist_df['ex_tz'] = hist_df['extraction_ts'].transform(lambda x: str(x).split('+')[1])
# is pd._libs.tslibs.timestamps.Timestamp]

In [114]:
dm(f"{hist_df['up_tz'].value_counts()}, {hist_df['ex_tz'].value_counts()}")

up_tz
08:00    8084696
Name: count, dtype: int64, ex_tz
08:00    8084696
Name: count, dtype: int64

In [117]:
hist_df.iloc[0:10]['last_update_ts']

0    2024-03-07 10:03:14+08:00
1    2024-03-07 15:09:17+08:00
2    2024-03-07 10:01:14+08:00
3    2024-03-07 10:01:14+08:00
4    2024-03-07 15:09:17+08:00
5    2024-03-07 09:47:14+08:00
6    2024-03-07 11:48:15+08:00
7    2024-03-07 15:28:19+08:00
8    2024-03-07 15:09:18+08:00
9    2024-03-06 23:56:14+08:00
Name: last_update_ts, dtype: object

In [118]:
pd.to_datetime(hist_df.iloc[0:10]['last_update_ts'], yearfirst=True, utc=True).dt.tz_convert(tz="Asia/Taipei")

0   2024-03-07 10:03:14+08:00
1   2024-03-07 15:09:17+08:00
2   2024-03-07 10:01:14+08:00
3   2024-03-07 10:01:14+08:00
4   2024-03-07 15:09:17+08:00
5   2024-03-07 09:47:14+08:00
6   2024-03-07 11:48:15+08:00
7   2024-03-07 15:28:19+08:00
8   2024-03-07 15:09:18+08:00
9   2024-03-06 23:56:14+08:00
Name: last_update_ts, dtype: datetime64[ns, Asia/Taipei]

In [16]:
hist_df['last_update_ts'] = pd.to_datetime(hist_df['last_update_ts'], yearfirst=True, utc=True).dt.tz_convert(tz="Asia/Taipei")
hist_df['extraction_ts'] = pd.to_datetime(hist_df['extraction_ts'], yearfirst=True, utc=True).dt.tz_convert(tz="Asia/Taipei")

In [17]:
hist_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3425022 entries, 0 to 3425021
Data columns (total 17 columns):
 #   Column          Dtype                      
---  ------          -----                      
 0   id              int64                      
 1   name            object                     
 2   type            int64                      
 3   space           int64                      
 4   full            int64                      
 5   empty           int64                      
 6   bike_yb2        int64                      
 7   bike_eyb        int64                      
 8   city            object                     
 9   area            object                     
 10  lat             float64                    
 11  lng             float64                    
 12  address         object                     
 13  is_open         int64                      
 14  place_id        float64                    
 15  last_update_ts  datetime64[ns, Asia/Taipei]
 16  

In [28]:
#intermediary checkpoint

concat_df.to_parquet("./youbike_dock_info_history_2024-03-01_2024-03-17_raw.parquet", index=False)

# Validate & clean the dataset

In [29]:
# Analyze Raw da
main_df = pd.read_parquet("./youbike_dock_info_history_2024-03-01_2024-03-17_raw.parquet")



In [30]:
display(main_df.shape)
main_df.info()

(11509718, 17)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11509718 entries, 0 to 11509717
Data columns (total 17 columns):
 #   Column          Dtype                      
---  ------          -----                      
 0   id              int64                      
 1   name            object                     
 2   type            int64                      
 3   space           int64                      
 4   full            int64                      
 5   empty           int64                      
 6   bike_yb2        int64                      
 7   bike_eyb        int64                      
 8   city            object                     
 9   area            object                     
 10  lat             float64                    
 11  lng             float64                    
 12  address         object                     
 13  is_open         int64                      
 14  place_id        float64                    
 15  last_update_ts  datetime64[ns, Asia/Taipei]
 16

**Columns are understood as follows:**
- id: unique identifier per bike station
- type: youbike type (1.0, 2.0)
- space: total available bike slots per station (= full + empty)
- full: nbr of occupied slots (= bike_yb2 + bike_eyb)
- empty: nbr of available slots for parking
- bike_yb2: count of YouBike 2.0 in occupied slots (see 'full')
- bike_eyb: count of E-Youbike 2.0 in occupied slots (see 'full')
- city: city where bike station is located
- area: city's district where bike station is located
- lat: latitude coordinate of bike station
- lng: longitude coordinate of bike station
- place_id: ??
- address: postal address of bike station
- is_open: ??
- last_updated_ts: timestamp of data in unix epoch
- extraction_ts: ts data was pulled from API

## Check types & missing values

In [129]:
display(main_df.info(show_counts=True))
display(main_df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8084696 entries, 0 to 8084695
Data columns (total 17 columns):
 #   Column          Non-Null Count    Dtype                      
---  ------          --------------    -----                      
 0   id              8084696 non-null  int64                      
 1   name            8084696 non-null  object                     
 2   type            8084696 non-null  int64                      
 3   space           8084696 non-null  int64                      
 4   full            8084696 non-null  int64                      
 5   empty           8084696 non-null  int64                      
 6   bike_yb2        8084696 non-null  int64                      
 7   bike_eyb        8084696 non-null  int64                      
 8   city            7957968 non-null  object                     
 9   area            8084696 non-null  object                     
 10  lat             8084696 non-null  float64                    
 11  lng        

None

id                      0
name                    0
type                    0
space                   0
full                    0
empty                   0
bike_yb2                0
bike_eyb                0
city               126728
area                    0
lat                     0
lng                     0
address                 0
is_open                 0
place_id          8084696
last_update_ts          0
extraction_ts           0
dtype: int64

**Observation**
- place_id contains no value
- city has empty values
- updated_at is an int64

In [31]:
# Drop empty cities
main_df.drop(main_df[main_df["city"].isna()].index, inplace=True) # Discard vals with null cities

#Drop youbikes of type 1 - because soon discontinued
main_df.drop(main_df[main_df["type"] != 2].index, inplace=True)

# Convert ts - Not necessary since using parquet
# main_df["last_update_ts"] = pd.to_datetime(main_df["last_update_ts"], yearfirst=True) 
# main_df["extraction_ts"] = pd.to_datetime(main_df["extraction_ts"], yearfirst=True) 

## Discard irrelevant features

In [32]:
main_df.drop(labels=["name", "area", "address", 'place_id'], axis=1, inplace=True)

In [33]:
dm("#### Any missing values left?")
display(main_df.isna().sum())

#### Any missing values left?

id                0
type              0
space             0
full              0
empty             0
bike_yb2          0
bike_eyb          0
city              0
lat               0
lng               0
is_open           0
last_update_ts    0
extraction_ts     0
dtype: int64

## Check assumptions

In [34]:
# Space = full + empty

space_uneq = main_df[main_df["space"] != (main_df["full"] + main_df["empty"])]


display(Markdown('#### Checking if space = full + empty'))
display(Markdown(f'**Rows where unequal:** {space_uneq.shape[0]}'))
display(Markdown(f'**As proportion of total dataset:** {space_uneq.shape[0] / main_df.shape[0]}')) #displays ratio of total bikes 
display(Markdown('**Excerpt output**'))
display(space_uneq.head(5)) # Displays where it is not true
dm(f'**conclusion:** Understanding the is_open code mapping would be useful to assess how to handle these records')

#### Checking if space = full + empty

**Rows where unequal:** 1081963

**As proportion of total dataset:** 0.10882570434364189

**Excerpt output**

,id,type,space,full,empty,bike_yb2,bike_eyb,city,lat,lng,is_open,last_update_ts,extraction_ts
1,500501081,2,25,0,0,0,0,新竹縣,24.82685,121.02695,0,2024-03-07 15:09:17+08:00,2024-03-07 18:25:10+08:00
4,500501077,2,25,0,0,0,0,新竹縣,24.82024,121.02175,0,2024-03-07 15:09:17+08:00,2024-03-07 18:25:10+08:00
7,500501075,2,20,0,0,0,0,新竹縣,24.81614,121.01710,0,2024-03-07 15:28:19+08:00,2024-03-07 18:25:10+08:00
8,500501073,2,18,0,0,0,0,新竹縣,24.82866,121.00324,0,2024-03-07 15:09:18+08:00,2024-03-07 18:25:10+08:00
10,501310001,2,20,0,0,0,0,臺南市,23.06361,120.38913,0,2024-03-06 21:06:15+08:00,2024-03-07 18:25:10+08:00


**conclusion:** Understanding the is_open code mapping would be useful to assess how to handle these records

In [35]:
dm(f'Inequality appears across all is_open, and mostly on 0 and 2. \n Proportion rows unequal vs total per is_open')
display(space_uneq["is_open"].value_counts() / main_df["is_open"].value_counts())

Inequality appears across all is_open, and mostly on 0 and 2. 
 Proportion rows unequal vs total per is_open

is_open
0    0.258989
1    0.099776
2    0.651100
3         NaN
4    0.050103
5    0.116701
Name: count, dtype: float64

In [36]:
dm('#### Check if full = bike_yb2 + bike_eyb')
full_uneq = main_df[main_df["full"] != (main_df["bike_yb2"] + main_df["bike_eyb"])]
dm(f'**Rows where unequal:** {full_uneq.shape[0]}')
dm(f'**Excerpt output where unequal**')
display(full_uneq.head(5))
dm(f'**Conclusion**: Assumption is valid')

#### Check if full = bike_yb2 + bike_eyb

**Rows where unequal:** 0

**Excerpt output where unequal**

,id,type,space,full,empty,bike_yb2,bike_eyb,city,lat,lng,is_open,last_update_ts,extraction_ts


**Conclusion**: Assumption is valid

## Check range and distribution of values

### Categorical

In [37]:
main_df_cat = main_df.select_dtypes(include=["category", "object"]) 
main_df_cat["type"] = main_df["type"]
main_df_cat["is_open"] = main_df["is_open"]

In [38]:
for col in main_df_cat.columns:
    unique_values = main_df[col].unique()
    print(f"Unique values in '{col}': {unique_values}")

Unique values in 'city': ['新竹縣' '苗栗縣' '臺南市' '台中市' '台北市' '新北市' '桃園市' '高雄市' '新竹科學工業園區' '屏東縣' '嘉義市'
 '新竹市']
Unique values in 'type': [2]
Unique values in 'is_open': [0 1 5 4 2 3]


In [39]:
#Map city to ascii chars
city_name_glossary = {"新北市": "XinBeiShi",
                      "台北市": "TaiBeiShi",
                      "台中市":"TaiZhongShi",
                      "高雄市":"GaoXiongShi",
                      "桃園市":"TaoYuanShi",
                      "臺南市": "TaiNanShi",
                      "嘉義市": "JiaYiShi",
                      "屏東縣": "PingDongXian",
                      "新竹市": "XinZhuShi",
                      "新竹縣": "XinZhuXian",
                      "苗栗縣" : "MiaoLiXian",
                      "新竹科學工業園區": "XinZhuKeXueGong"}  

#TODO Need to catch KeyError and set default value. 
main_df["city"] = main_df.apply(lambda x: city_name_glossary[x["city"]], axis=1)


### Numerical

In [40]:
main_df_num = main_df.drop(labels=main_df_cat.columns, axis=1)
main_df_num['last_update_ts'] = main_df['last_update_ts'].copy(deep=True)
main_df_num['extraction_ts'] = main_df['extraction_ts'].copy(deep=True)

In [41]:
pd.set_option('display.float_format', '{:.2f}'.format)


In [42]:
main_df_num.describe(include='all')

,id,space,full,empty,bike_yb2,bike_eyb,lat,lng,last_update_ts,extraction_ts
count,9942164.00,9942164.00,9942164.00,9942164.00,9942164.00,9942164.00,9942164.00,9942164.00,9942164,9942164
mean,500639066.25,20.96,7.25,13.25,6.92,0.34,24.11,120.56,2024-03-10 07:09:51.085953280+08:00,2024-03-11 18:30:09.624945664+08:00
min,500101001.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2024-01-14 01:18:15+08:00,2024-03-01 04:18:31+08:00
25%,500207013.00,14.00,3.00,7.00,2.00,0.00,23.01,120.38,2024-03-08 07:07:19+08:00,2024-03-08 13:25:29+08:00
50%,500602029.00,17.00,6.00,11.00,5.00,0.00,24.42,120.83,2024-03-12 17:21:39+08:00,2024-03-12 22:00:59+08:00
75%,501203106.00,24.00,10.00,16.00,9.00,0.00,25.03,121.49,2024-03-15 08:11:19+08:00,2024-03-15 11:30:56+08:00
max,508201041.00,99.00,99.00,99.00,99.00,62.00,25.29,122.00,2024-03-17 23:49:18+08:00,2024-03-17 23:50:55+08:00
std,741750.68,11.96,7.05,10.36,6.96,1.22,1.65,6.83,NaN,NaN


**Observations about range and distribution**
- Empty has negative values
- Space min value is zero
- Some long/lat coordinates are outside of Taiwan
- Some ts shows stale data (many days older than latest ts)
- Rest is according to expectations

In [43]:
dm("""
**Dropping:**
- Stations with negative empty values. (= Available spaces cannot be negative.)
- Stations without space are irrelevant
- Stations located outside of Taiwan (these are test locations. Confirmed by not being displayed on YouBikes official map)
- Stations with last_update_ts older than the first day of this historical dataset. (Stale data is useless)
""")
dm(f"Row counts before: {main_df.shape[0]}")
discard_mask = (
    (main_df["empty"] < 0)
    | (main_df["space"] < 1) # Stations without space are irrelevant
    | (main_df["lat"] < 21.89) #southermost lat of Taiwan's main island
    | (main_df["lng"] < 120) #westernmost lng of Taiwan's main island
    | (main_df["last_update_ts"] < pd.to_datetime("2024-02-16").tz_localize(tz='Asia/Taipei'))
)
main_df.drop(main_df[discard_mask == True].index, inplace=True)

dm(f"Row counts after: {main_df.shape[0]}")
#TODO: bound to the easter and northernmost points too? 


**Dropping:**
- Stations with negative empty values. (= Available spaces cannot be negative.)
- Stations without space are irrelevant
- Stations located outside of Taiwan (these are test locations. Confirmed by not being displayed on YouBikes official map)
- Stations with last_update_ts older than the first day of this historical dataset. (Stale data is useless)


Row counts before: 9942164

Row counts after: 9677042

## Check uniqueness of observations

In [44]:
dm("#### Unique key is (extraction_ts, id)") 

main_df['identical_key_cnt'] = main_df.groupby(['extraction_ts', 'id']).transform('size')
display(main_df['identical_key_cnt'].value_counts())
dm("key_cnt = 1 means no duplicates")

#### Unique key is (extraction_ts, id)

identical_key_cnt
1    9677042
Name: count, dtype: int64

key_cnt = 1 means no duplicates

In [45]:
dm("### Does each station have a unique record per extraction? (uniqueness on position (= Lat / Lng)")

main_df["pos"] = main_df.apply(lambda x: (x["lat"], x["lng"]), axis=1)
main_df["duplic_pos"] = main_df.groupby(['pos', 'extraction_ts']).transform('size')

display(main_df[main_df["duplic_pos"] > 1].head(4))
dm("**Observation:** Some Lat / Lng are recorded twice within the same extraction")
display(main_df["duplic_pos"].value_counts())
dm("<br>Filtering for those positions which are recorded twice returns ")
display(main_df[main_df["duplic_pos"] > 1]["pos"].value_counts())
dm("**Conclusion**: For the moment, any duplicated station (identified by using pos) per extraction will be dismissed")

### Does each station have a unique record per extraction? (uniqueness on position (= Lat / Lng)

,id,type,space,full,empty,bike_yb2,bike_eyb,city,lat,lng,is_open,last_update_ts,extraction_ts,identical_key_cnt,pos,duplic_pos
5504,500199002,2,17,0,17,0,0,TaiBeiShi,25.00,121.54,5,2024-03-07 18:19:20+08:00,2024-03-07 18:25:10+08:00,1,"(24.99609, 121.54284)",2
6446,500105068,2,10,1,9,1,0,TaiBeiShi,25.00,121.54,1,2024-03-07 18:23:14+08:00,2024-03-07 18:25:10+08:00,1,"(24.99609, 121.54284)",2
13387,500199002,2,17,0,17,0,0,TaiBeiShi,25.00,121.54,5,2024-03-01 11:33:20+08:00,2024-03-01 11:57:38+08:00,1,"(24.99609, 121.54284)",2
14329,500105068,2,10,0,10,0,0,TaiBeiShi,25.00,121.54,1,2024-03-01 11:48:14+08:00,2024-03-01 11:57:38+08:00,1,"(24.99609, 121.54284)",2


**Observation:** Some Lat / Lng are recorded twice within the same extraction

duplic_pos
1    9672656
2       4386
Name: count, dtype: int64

<br>Filtering for those positions which are recorded twice returns 

pos
(24.99609, 121.54284)    2906
(25.03062, 121.49021)     756
(25.07965, 121.54148)     724
Name: count, dtype: int64

**Conclusion**: For the moment, any duplicated station (identified by using pos) per extraction will be dismissed

In [46]:
dm("Dropping stations with duplicated pos, extraction_ts")
dm(f"Row count before: {main_df.shape[0]}")

main_df.drop(main_df[main_df["duplic_pos"] > 1].index, inplace=True)

dm(f"Row count after: {main_df.shape[0]}")

Dropping stations with duplicated pos, extraction_ts

Row count before: 9677042

Row count after: 9672656

In [47]:
#Last visual check before dl
#Checkpoint
base_df = main_df[['id', 'space', 'full', 'empty', 'city', 
                   'lat', 'lng', 'last_update_ts', 'extraction_ts']].copy(deep=True)
base_df

,id,space,full,empty,city,lat,lng,last_update_ts,extraction_ts
1,500501081,25,0,0,XinZhuXian,24.83,121.03,2024-03-07 15:09:17+08:00,2024-03-07 18:25:10+08:00
4,500501077,25,0,0,XinZhuXian,24.82,121.02,2024-03-07 15:09:17+08:00,2024-03-07 18:25:10+08:00
7,500501075,20,0,0,XinZhuXian,24.82,121.02,2024-03-07 15:28:19+08:00,2024-03-07 18:25:10+08:00
8,500501073,18,0,0,XinZhuXian,24.83,121.00,2024-03-07 15:09:18+08:00,2024-03-07 18:25:10+08:00
10,501310001,20,0,0,TaiNanShi,23.06,120.39,2024-03-06 21:06:15+08:00,2024-03-07 18:25:10+08:00
...,...,...,...,...,...,...,...,...,...
11508692,500101012,8,1,7,TaiBeiShi,25.02,121.53,2024-03-17 14:27:18+08:00,2024-03-17 14:40:53+08:00
11508693,500101013,9,6,3,TaiBeiShi,25.03,121.54,2024-03-17 14:39:19+08:00,2024-03-17 14:40:53+08:00
11508694,500101014,11,9,2,TaiBeiShi,25.02,121.53,2024-03-17 14:39:19+08:00,2024-03-17 14:40:53+08:00
11508695,500101015,14,2,11,TaiBeiShi,25.02,121.55,2024-03-17 14:33:18+08:00,2024-03-17 14:40:53+08:00


In [48]:
base_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9672656 entries, 1 to 11508696
Data columns (total 9 columns):
 #   Column          Dtype                      
---  ------          -----                      
 0   id              int64                      
 1   space           int64                      
 2   full            int64                      
 3   empty           int64                      
 4   city            object                     
 5   lat             float64                    
 6   lng             float64                    
 7   last_update_ts  datetime64[ns, Asia/Taipei]
 8   extraction_ts   datetime64[ns, Asia/Taipei]
dtypes: datetime64[ns, Asia/Taipei](2), float64(2), int64(4), object(1)
memory usage: 738.0+ MB


In [49]:
base_df.to_parquet("../tmp_data/clean_data/youbike_dock_info_history_2024-03-01_2024-03-17.parquet", index=False)

# Analyze Data


In [ ]:
base_df = pd.read_parquet("youbike_dock_info_history_2024-02-15_2024-03-14_raw.parquet")

In [ ]:
rand_id = base_df['id'].sample(1, ignore_index=True)[0]
print(rand_id)

In [ ]:
eg_id = rand_id
# base_df.sort_values(by=["id", "last_update_ts"]).head(50)
s_1 = base_df[base_df["id"] == eg_id] \
    .sort_values(by=["id", "extraction_ts"], ascending=False) \
    .copy(deep=True).reset_index(drop=True)
# s_1.drop_duplicates(subset="last_update_ts", inplace=True)
s_1["pct_full"] = s_1["full"] / s_1["space"]

## Apply rolling average 

s_1["30m_rol_avg_pct_full"] = s_1['pct_full'].rolling(window=3).mean() #Assumes each record is 10 mins

In [ ]:
sns.relplot(data=s_1.iloc[:632], kind="line", x="extraction_ts", y="30m_rol_avg_pct_full", aspect=3)

plt.xticks(rotation=20)
plt.ylim(0, 1)
plt.show()

In [ ]:
s_1.head(10)